In [3]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import tensorflow as tf
from tensorflow.keras import layers
from transformers import TFBertModel

# Define the custom BertLayer
class BertLayer(layers.Layer):
    def __init__(self, pretrained_model_name, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return outputs.last_hidden_state


In [8]:
from transformers import BertTokenizer
import numpy as np

class MBTIPredictor:
    def __init__(self, model_path, tokenizer_name='bert-large-uncased', max_length=128):
        # Load the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length
        
        # Load the model
        self.model = tf.keras.models.load_model(
            model_path,
            custom_objects={'BertLayer': BertLayer}  
        )
        
        # Define the MBTI traits
        self.traits = ['IE', 'NS', 'TF', 'JP']

    def tokenize_text(self, text):
        # Tokenize the input text
        tokenized_data = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='np'
        )
        return {
            "input_word_ids": tokenized_data['input_ids'],
            "attention_mask": tokenized_data['attention_mask'],
            "token_type_ids": tokenized_data['token_type_ids']
        }

    def interpret_predictions(self, predictions):
        # Interpret predictions to construct MBTI type
        personality = []
        for trait, output in zip(self.traits, predictions):
            score = output[0]  # Single input prediction
            if score > 0.5:
                personality.append(trait[0])  # First letter (e.g., 'I' for 'IE')
            else:
                personality.append(trait[1])  # Second letter (e.g., 'E' for 'IE')
        return ''.join(personality)

    def predict(self, text):
        # Full pipeline: tokenize, predict, interpret
        tokenized_input = self.tokenize_text(text)
        predictions = self.model.predict(tokenized_input)
        return self.interpret_predictions(predictions)

# Usage
predictor = MBTIPredictor(model_path="bert_model.keras")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [14]:
input_text = (
    """Getting started with data science and applying machine learning has never been as simple as it is now. There are many free and paid online tutorials and courses out there to help you to get started. I’ve recently started to learn, play, and work on Data Science & Machine Learning on Kaggle.com. In this brief post, I’d like to share my experience with the Kaggle Python Docker image, which simplifies the Data Scientist’s life.
Awesome #AWS monitoring introduction.
HPE Software (now @MicroFocusSW) won the platinum reader's choice #ITAWARDS 2017 in the new category #CloudMonitoring
Certified as AWS Certified Solutions Architect 
Hi, please have a look at my Udacity interview about online learning and machine learning,
Very interesting to see the  lessons learnt during the HP Operations Orchestration to CloudSlang journey. http://bit.ly/1Xo41ci 
I came across a post on devopsdigest.com and need your input: “70% DevOps organizations Unhappy with DevOps Monitoring Tools”
In a similar investigation I found out that many DevOps organizations use several monitoring tools in parallel. Senu, Nagios, LogStach and SaaS offerings such as DataDog or SignalFX to name a few. However, one element is missing: Consolidation of alerts and status in a single pane of glass, which enables fast remediation of application and infrastructure uptime and performance issues.
Sure, there are commercial tools on the market for exactly this use case but these tools are not necessarily optimized for DevOps.
So, here my question to you: In your DevOps project, have you encountered that the lack of consolidation of alerts and status is a real issue? If yes, how did you approach the problem? Or is an ChatOps approach just right?
You will probably hear more and more about ChatOps - at conferences, DevOps meet-ups or simply from your co-worker at the coffee station. ChatOps is a term and concept coined by GitHub. It's about the conversation-driven development, automation, and operations.
Now the question is: why and how would I, as an ops-focused engineer, implement and use ChatOps in my organization? The next question then is: How to include my tools into the chat conversation?
Let’s begin by having a look at a use case. The Closed Looped Incidents Process (CLIP) can be rejuvenated with ChatOps. The work from the incident detection runs through monitoring until the resolution of issues in your application or infrastructure can be accelerated with improved, cross-team communication and collaboration.
In this blog post, I am going to describe and share my experience with deploying HP Operations Manager i 10.0 (OMi) on HP Helion Public Cloud. An Infrastructure as a Service platform such as HP Helion Public Cloud Compute is a great place to quickly spin-up a Linux server and install HP Operations Manager i for various use scenarios. An example of a good use case is monitoring workloads across public clouds such as AWS and Azure.
"""
)

# Get MBTI personality type
predicted_mbti = predictor.predict(input_text)
print(f"Predicted MBTI Personality: {predicted_mbti}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
Predicted MBTI Personality: INFP
